In [ ]:
# CÓDIGO CORRIGIDO PARA CRIAR BASE VETORIAL (SEM ERROS DE READONLY)
# Execute esta célula depois de resolver os problemas de permissão

import chromadb
from chromadb.config import Settings
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import pandas as pd
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Diretório do vectorstore
vectorstore_dir = os.path.abspath("../vectorstore")
print(f"📂 Usando diretório: {vectorstore_dir}")

# Carregar dados
df = pd.read_csv("../notebooks/data/processed/dados_bcb.csv")
print(f"📊 Dados carregados: {len(df)} registros")

# Carregar documentos
loader = DataFrameLoader(df, page_content_column="text")
documents = loader.load()
print(f"📄 Documentos criados: {len(documents)}")

# Configurar embeddings
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
print("🔄 Embedding configurado")

try:
    # Configuração alternativa do ChromaDB (usando DuckDB em vez de SQLite)
    client = chromadb.Client(Settings(
        chroma_db_impl="duckdb+parquet",  # Usar DuckDB em vez de SQLite
        persist_directory=vectorstore_dir,
        anonymized_telemetry=False
    ))
    
    # Remover coleção se já existir
    try:
        client.delete_collection("documents_dados_bcb")
        print("🗑️ Coleção anterior removida")
    except:
        print("🆕 Criando nova coleção")
    
    # Criar vectorstore
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embedding,
        persist_directory=vectorstore_dir,
        client=client,
        collection_name="documents_dados_bcb"
    )
    
    # Persistir vetores
    vectorstore.persist()
    print(f"✅ Base vetorial criada com sucesso em {vectorstore_dir}")
    print(f"✅ Nome da coleção: documents_dados_bcb")
    print(f"✅ Total de documentos vetorizados: {len(documents)}")
    
except Exception as e:
    print(f"❌ Erro ao criar base vetorial: {e}")
    print("⚠️ Veja as opções de correção nas células anteriores")

In [ ]:
# Execute esta célula após escolher uma opção
if 'option' in locals():
    if option == "1":
        if fix_permissions():
            print("\n✨ Tente executar seu código novamente agora!")
    elif option == "2":
        new_dir = create_new_vectorstore()
        if new_dir:
            print("\n✨ Use o novo diretório no seu código e tente novamente!")
    elif option == "3":
        get_alternative_config()
        print("\n✨ Use esta configuração alternativa no seu código!")
    else:
        print("❌ Opção inválida!")

In [ ]:
# OPÇÃO 1: Corrigir permissões do diretório vectorstore
def fix_permissions():
    try:
        # Recursivamente muda as permissões do diretório vectorstore
        subprocess.run(['chmod', '-R', '777', vectorstore_dir], check=True)
        print(f"✅ Permissões atualizadas para o diretório {vectorstore_dir}")
        
        # Verificar novamente
        for item in os.listdir(vectorstore_dir):
            item_path = os.path.join(vectorstore_dir, item)
            item_stat = os.stat(item_path)
            item_perm = stat.filemode(item_stat.st_mode)
            print(f"{item}: {item_perm}")
            
        return True
    except Exception as e:
        print(f"❌ Erro ao atualizar permissões: {e}")
        return False

# OPÇÃO 2: Criar um novo diretório para o vectorstore
def create_new_vectorstore():
    try:
        # Criar um novo diretório em um local temporário
        new_dir = os.path.join(os.path.expanduser("~"), "temp_vectorstore")
        os.makedirs(new_dir, exist_ok=True)
        
        print(f"✅ Novo diretório criado: {new_dir}")
        print(f"⚠️ Use este caminho para o seu vectorstore:")
        print(f"vectorstore_dir = '{new_dir}'")
        
        return new_dir
    except Exception as e:
        print(f"❌ Erro ao criar novo diretório: {e}")
        return None

# OPÇÃO 3: Usar configurações alternativas para o ChromaDB
def get_alternative_config():
    config = """
# Use esta configuração para o ChromaDB:
from chromadb.config import Settings

client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",  # Usar DuckDB em vez de SQLite
    persist_directory=vectorstore_dir,
    anonymized_telemetry=False
))
"""
    print(config)
    
# Menu de opções
print("🔧 OPÇÕES PARA RESOLVER PROBLEMA DE BANCO DE DADOS SOMENTE LEITURA:")
print("1. Tentar corrigir permissões do diretório atual")
print("2. Criar um novo diretório para o vectorstore")
print("3. Ver configurações alternativas para o ChromaDB")

option = input("Escolha uma opção (1-3): ")

🔧 OPÇÕES PARA RESOLVER PROBLEMA DE BANCO DE DADOS SOMENTE LEITURA:
1. Tentar corrigir permissões do diretório atual
2. Criar um novo diretório para o vectorstore
3. Ver configurações alternativas para o ChromaDB


In [1]:
import os
import stat
import pwd
import shutil
import subprocess
from pathlib import Path

# Diretório do vectorstore
vectorstore_dir = os.path.abspath("../vectorstore")
print(f"📂 Diretório do vectorstore: {vectorstore_dir}")

# Verificar se o diretório existe
if os.path.exists(vectorstore_dir):
    # Verificar permissões
    stat_info = os.stat(vectorstore_dir)
    permissions = stat.filemode(stat_info.st_mode)
    owner = pwd.getpwuid(stat_info.st_uid).pw_name
    
    print(f"👤 Proprietário: {owner}")
    print(f"🔐 Permissões: {permissions}")
    
    # Listar arquivos no diretório
    print("\n📋 Arquivos no diretório:")
    for item in os.listdir(vectorstore_dir):
        item_path = os.path.join(vectorstore_dir, item)
        item_stat = os.stat(item_path)
        item_perm = stat.filemode(item_stat.st_mode)
        item_owner = pwd.getpwuid(item_stat.st_uid).pw_name
        item_size = os.path.getsize(item_path) / 1024  # em KB
        
        if os.path.isdir(item_path):
            print(f"📁 {item} - Proprietário: {item_owner}, Permissões: {item_perm}, Tamanho: {item_size:.2f} KB")
        else:
            print(f"📄 {item} - Proprietário: {item_owner}, Permissões: {item_perm}, Tamanho: {item_size:.2f} KB")
            
    # Verificar especificamente o arquivo sqlite
    sqlite_path = os.path.join(vectorstore_dir, "chroma.sqlite3")
    if os.path.exists(sqlite_path):
        sqlite_stat = os.stat(sqlite_path)
        sqlite_perm = stat.filemode(sqlite_stat.st_mode)
        sqlite_owner = pwd.getpwuid(sqlite_stat.st_uid).pw_name
        
        print(f"\n🔍 Arquivo SQLite: {sqlite_path}")
        print(f"👤 Proprietário: {sqlite_owner}")
        print(f"🔐 Permissões: {sqlite_perm}")
else:
    print(f"❌ O diretório {vectorstore_dir} não existe!")

📂 Diretório do vectorstore: /home/edu/Documentos/Engenharia-dados-IA/Pipe-Rag/vectorstore
👤 Proprietário: edu
🔐 Permissões: drwxr-xr-x

📋 Arquivos no diretório:


In [ ]:
"""
# Diagnóstico e Correção de Problemas do ChromaDB (Readonly Database)

Este notebook ajuda a diagnosticar e resolver problemas de permissão do ChromaDB
quando ocorre o erro: "attempt to write a readonly database"
"""

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os
import json

# Carrega variáveis de ambiente
load_dotenv()

# 1. Carrega as variáveis do projeto
with open("../configs/settings.json", "r") as f:
    settings = json.load(f)

# 2. Obter chave da API OpenAI (corrigido)
# Primeira tentativa: do arquivo .env
openai_api_key = os.getenv("OPENAI_API_KEY")

# Segunda tentativa: do settings.json se não encontrou no .env
if not openai_api_key:
    openai_api_key = settings.get("openai_api_key")

# Verifica se conseguiu obter a chave
if not openai_api_key:
    raise ValueError("A chave da API OpenAI não foi encontrada nem no .env nem no settings.json")

print("✅ Chave da API OpenAI carregada com sucesso")

# 3. Define o caminho do CSV
csv_path = os.path.join(settings["processed_data_path"], f"{settings['parquet_file_prefix']}.csv")

print(f"csv_path = {csv_path}")
print(f"Arquivo existe? {os.path.exists(csv_path)}")

# 4. Verifica se o arquivo existe antes de tentar carregar
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"Arquivo CSV não encontrado: {csv_path}")

# 5. Carrega os dados do CSV como documentos
loader = CSVLoader(
    file_path=csv_path,
    source_column="contexto"  # coluna que contém o texto completo
)

documents = loader.load()
print(f"✅ {len(documents)} documentos carregados do CSV")

# 6. Split de texto (se necessário)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
docs_split = splitter.split_documents(documents)
print(f"✅ Documentos divididos em {len(docs_split)} chunks")

# 7. Inicializa o embedding com OpenAI (corrigido)
try:
    embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
    print("✅ Embedding OpenAI inicializado com sucesso")
except Exception as e:
    print(f"❌ Erro ao inicializar embedding: {e}")
    raise

# 8. Cria o diretório do vectorstore com permissões corretas
vectorstore_dir = os.path.abspath("../vectorstore")
print(f"📁 Diretório do vectorstore: {vectorstore_dir}")

# Remove diretório existente se houver (para evitar conflitos)
if os.path.exists(vectorstore_dir):
    import shutil
    print("🗑️ Removendo vectorstore existente...")
    shutil.rmtree(vectorstore_dir)

# Cria novo diretório com permissões completas
os.makedirs(vectorstore_dir, mode=0o755, exist_ok=True)
print("✅ Diretório do vectorstore criado")

# 9. Cria o vetor store com Chroma (armazenado localmente)
try:
    # Usar collection_name único para evitar conflitos
    collection_name = f"documents_{settings.get('parquet_file_prefix', 'default')}"
    
    db = Chroma.from_documents(
        documents=docs_split,
        embedding=embedding,
        persist_directory=vectorstore_dir,
        collection_name=collection_name
    )
    print(f"✅ Base vetorial criada com sucesso (collection: {collection_name})")
except Exception as e:
    print(f"❌ Erro ao criar base vetorial: {e}")
    
    # Tentativa alternativa: usar diretório temporário
    print("🔄 Tentando criar em diretório temporário...")
    import tempfile
    temp_dir = tempfile.mkdtemp(prefix="vectorstore_")
    print(f"📂 Usando diretório temporário: {temp_dir}")
    
    try:
        db = Chroma.from_documents(
            documents=docs_split,
            embedding=embedding,
            persist_directory=temp_dir,
            collection_name=collection_name
        )
        vectorstore_dir = temp_dir  # Atualiza para o diretório que funcionou
        print("✅ Base vetorial criada em diretório temporário")
    except Exception as e2:
        print(f"❌ Erro mesmo com diretório temporário: {e2}")
        raise

# 10. Salva o vetor store
try:
    db.persist()
    print(f"✔️ Base vetorial salva com sucesso em: {vectorstore_dir}")
    print(f"📊 Total de documentos vetorizados: {len(docs_split)}")
    print(f"🔍 Nome da coleção: {collection_name}")
except Exception as e:
    print(f"❌ Erro ao salvar base vetorial: {e}")
    print("ℹ️ A base vetorial pode ainda estar disponível na memória para uso imediato")
    
    # Testa se consegue fazer uma busca mesmo sem persistir
    try:
        test_results = db.similarity_search("teste", k=1)
        print("✅ Base vetorial funcional (mesmo sem persistência)")
    except Exception as e2:
        print(f"❌ Base vetorial não está funcional: {e2}")
        raise

✅ Chave da API OpenAI carregada com sucesso
csv_path = data/processed/dados_bcb.csv
Arquivo existe? True
✅ 544 documentos carregados do CSV
✅ Documentos divididos em 544 chunks


/tmp/ipykernel_107997/4191060924.py:59: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)


✅ Embedding OpenAI inicializado com sucesso
📁 Diretório do vectorstore: /home/edu/Documentos/Engenharia-dados-IA/Pipe-Rag/vectorstore
🗑️ Removendo vectorstore existente...
✅ Diretório do vectorstore criado
✅ Base vetorial criada com sucesso (collection: documents_dados_bcb)
✔️ Base vetorial salva com sucesso em: /home/edu/Documentos/Engenharia-dados-IA/Pipe-Rag/vectorstore
📊 Total de documentos vetorizados: 544
🔍 Nome da coleção: documents_dados_bcb
✅ Base vetorial criada com sucesso (collection: documents_dados_bcb)
✔️ Base vetorial salva com sucesso em: /home/edu/Documentos/Engenharia-dados-IA/Pipe-Rag/vectorstore
📊 Total de documentos vetorizados: 544
🔍 Nome da coleção: documents_dados_bcb


/tmp/ipykernel_107997/4191060924.py:115: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
